In [ ]:
# Ad: Nahit Furkan 
# Soyad: Öznamlı
# Okul Numarası: 2212721020
# GitHub Repo Bağlantısı: https://github.com/FurkanOznamli/MLP_Flask

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import statsmodels.api as sm
import pickle

# Grafikler için (opsiyonel, ama güzel görünür)
sns.set(style="whitegrid")


In [ ]:
# Veri setini okuma
df = pd.read_csv("insurance.csv")

# Kolon isimlerini Türkçeye çevirme
df = df.rename(columns={
    "age": "yas",
    "sex": "cinsiyet",
    "bmi": "vucut_kitle_indeksi",
    "children": "cocuk_sayisi",
    "smoker": "sigara_iciyor_mu",
    "region": "bolge",
    "charges": "sigorta_ucreti"
})

# İlk 5 satırı göster
df.head()


,yas,cinsiyet,vucut_kitle_indeksi,cocuk_sayisi,sigara_iciyor_mu,bolge,sigorta_ucreti
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
#Eksik veri var mı kontrol ediyorum
df.info()
df.isnull().sum()
#Datasetde eksik veri yok bu yüzden bir işlem yapmayacağım eğer eksik veri olsaydı ortalama alarak dolduracakdım.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   yas                  1338 non-null   int64  
 1   cinsiyet             1338 non-null   object 
 2   vucut_kitle_indeksi  1338 non-null   float64
 3   cocuk_sayisi         1338 non-null   int64  
 4   sigara_iciyor_mu     1338 non-null   object 
 5   bolge                1338 non-null   object 
 6   sigorta_ucreti       1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,0
yas,0
cinsiyet,0
vucut_kitle_indeksi,0
cocuk_sayisi,0
sigara_iciyor_mu,0
bolge,0
sigorta_ucreti,0


In [ ]:
# Sayısal değişkenler
numeric_cols = ["yas", "vucut_kitle_indeksi", "cocuk_sayisi"]

# Kategorik değişkenler
categorical_cols = ["cinsiyet", "sigara_iciyor_mu", "bolge"]

numeric_cols, categorical_cols


(['yas', 'vucut_kitle_indeksi', 'cocuk_sayisi'],
 ['cinsiyet', 'sigara_iciyor_mu', 'bolge'])

In [ ]:
X = df.drop("sigorta_ucreti", axis=1)
y = df["sigorta_ucreti"]

X.head(), y.head()


(   yas cinsiyet  vucut_kitle_indeksi  cocuk_sayisi sigara_iciyor_mu      bolge
 0   19   female               27.900             0              yes  southwest
 1   18     male               33.770             1               no  southeast
 2   28     male               33.000             3               no  southeast
 3   33     male               22.705             0               no  northwest
 4   32     male               28.880             0               no  northwest,
 0    16884.92400
 1     1725.55230
 2     4449.46200
 3    21984.47061
 4     3866.85520
 Name: sigorta_ucreti, dtype: float64)

In [ ]:
# Cinsiyet, sigara kullanımı ve bölge sıralı kategoriler olmadığı için One-Hot Encoding tercih edildi;
#Bu yöntem modelin kategoriler arasında yanlış sıralama yapmasını engeller.
ct = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(drop="first"), categorical_cols)
    ],
    remainder="passthrough"
)

ct


ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(drop='first'),
                                 ['cinsiyet', 'sigara_iciyor_mu', 'bolge'])])

In [ ]:
# One-Hot Encoding'i X üzerine uygulayalım
X_encoded = ct.fit_transform(X)

# Statsmodels OLS için DataFrame'e dönüştürme
X_be = pd.DataFrame(X_encoded)

# OLS regresyonu için sabit terim ekleme
X_be = sm.add_constant(X_be)

X_be.head()


,const,0,1,2,3,4,5,6,7
0,1.0,0.0,1.0,0.0,0.0,1.0,19.0,27.900,0.0
1,1.0,1.0,0.0,0.0,1.0,0.0,18.0,33.770,1.0
2,1.0,1.0,0.0,0.0,1.0,0.0,28.0,33.000,3.0
3,1.0,1.0,0.0,1.0,0.0,0.0,33.0,22.705,0.0
4,1.0,1.0,0.0,1.0,0.0,0.0,32.0,28.880,0.0


In [ ]:
cols = list(range(X_be.shape[1]))

while True:
    X_opt = X_be.iloc[:, cols]
    model = sm.OLS(y, X_opt).fit()
    p_values = model.pvalues

    max_p = p_values.max()
    max_p_index = p_values.idxmax()

    # idxmax bazen float/string döndürür → kolondaki gerçek indexi bul
    true_index = cols[p_values.argmax()]

    if max_p > 0.05:
        cols.remove(true_index)
    else:
        break

model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         sigorta_ucreti   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     998.1
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        11:34:46   Log-Likelihood:                -13551.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1333   BIC:                         2.714e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -1.21e+04    941.984    -12.848      0.000    -1.4e+04   -1.03e+04
1           2.381e+04    411.220     57.904      0.000     2.3e+04    2.46e+04
5            257.8495     11.896     21.675      0.000     234.512     281.187
6            321.8514     27.378     11.756      0.000     268.143     375.559
7            473.5023    137.792      3.436      0.001     203.190     743.814
==============================================================================
Omnibus:                      301.480   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              722.157
Skew:                           1.215   Prob(JB):                    1.53e-157
Kurtosis:                       5.654   Cond. No.                         292.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# One-Hot işleminden sonra oluşan X_encoded matrisini kullanıyoruz
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# Regresyon modeli
reg = LinearRegression()
reg.fit(X_train, y_train)


LinearRegression()

In [ ]:
y_pred = reg.predict(X_test)

print("R2 Skoru:", r2_score(y_test, y_pred)) # R² skorunun 0.78 çıkması, modelin sigorta ücretini oldukça iyi açıkladığını gösteriyor. Yani değişkenler ücretin yaklaşık %78’ini açıklayabiliyor.
print("MAE:", mean_absolute_error(y_test, y_pred)) # MAE yaklaşık 4181 çıktı. Bu, modelin yaptığı tahminlerin ortalama olarak gerçek değerden 4181 kadar sapma yaptığını gösteriyor.
print("MSE:", mean_squared_error(y_test, y_pred)) # MSE değeri 3.35e7 seviyesinde. Büyük hataları daha fazla cezalandırdığı için bu değer doğal olarak yüksek görünür.


R2 Skoru: 0.7835929767120722
MAE: 4181.194473753643
MSE: 33596915.85136148


In [ ]:
with open("sigorta_ucreti_tahmin_modeli.pkl", "wb") as f:
    pickle.dump((reg, ct), f)


In [ ]:
import sklearn
sklearn.__version__


'1.6.1'